In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=6da5eed5a69811093f67aa4f68497b46496d177fa37b229ed849142772847dc3
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
! git clone https://github.com/facebookresearch/fastText.git
! cd fastText

Cloning into 'fastText'...
remote: Enumerating objects: 3986, done.
remote: Counting objects: 100% (1014/1014), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 3986 (delta 888), reused 854 (delta 830), pack-reused 2972
Receiving objects: 100% (3986/3986), 8.29 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (2526/2526), done.


In [ ]:
import pandas as pd

df= pd.read_csv("/content/drive/MyDrive/spacy_textcat/dstool_classifier.csv")
print(df.shape)
df.head(3)

(534, 2)


,text,label
0,early departure fee,"""early departure fees"""
1,rollaway bed rental,"""rollawayscribs"""
2,pet charge,"""pet fees"""


In [ ]:
df.dropna(inplace=True)
df.shape

(534, 2)

In [ ]:
df.label.unique()


array(['"early departure fees"', '"rollawayscribs"', '"pet fees"',
       '"banquet food revenue breakfast"', '"banquet food revenue lunch"',
       '"banquet food revenue dinner"',
       '"restaurant one beverage sales wine"',
       '"restaurant one beverage sales liquor"',
       '"banquet beverage revenue beverage"',
       '"telephone income local long distance"', '"commission income"',
       '"smoking fees"', '"late checkout fees"', '"out of order rooms"',
       '"vacant rooms"', '"gift shop sales"', '"miscellaneous income"',
       '"transient discount"', '"group corporate"', '"group government"',
       '"group smerf"', '"transient corporate negotiate"',
       '"banquet other miscellaneous revenue"', '"internet income"',
       '"cancellation fees"', '"guaranteed no show rooms"',
       '"transient retail"', '"rooms city tax"', '"rooms sales tax"',
       '"comp meal allowance room service"',
       '"payments received towards city ledger"',
       '"rooms county occupancy 

In [ ]:
a = df.label.unique()
len(a)

72

In [ ]:
df['label'] = '__label__' + df['label'].astype(str)
df.head(5)

,text,label
0,early departure fee,"__label__""early departure fees"""
1,rollaway bed rental,"__label__""rollawayscribs"""
2,pet charge,"__label__""pet fees"""
3,pet cleaning fee,"__label__""pet fees"""
4,banquet breakfast,"__label__""banquet food revenue breakfast"""


In [ ]:
def add_label_prefix(label_str):
    words = label_str.split()
    labeled_words = ['_label_' + word for word in words]
    return ' '.join(labeled_words)

# Add '_label_' prefix to each word in the 'label' column
df['label'] = df['label'].apply(add_label_prefix)

df.head(5)

In [ ]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [ ]:
df['label_text'] = df['label'] + ' ' + df['text']
df.head(5)

,text,label,label_text
0,early departure fee,"__label__""early departure fees""","__label__""early departure fees"" early departur..."
1,rollaway bed rental,"__label__""rollawayscribs""","__label__""rollawayscribs"" rollaway bed rental"
2,pet charge,"__label__""pet fees""","__label__""pet fees"" pet charge"
3,pet cleaning fee,"__label__""pet fees""","__label__""pet fees"" pet cleaning fee"
4,banquet breakfast,"__label__""banquet food revenue breakfast""","__label__""banquet food revenue breakfast"" banq..."


In [ ]:
df.to_csv('classification_data.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
train.shape, test.shape

((427, 3), (107, 3))

In [ ]:
df

,text,label,label_text
0,early departure fee,"__label__""early departure fees""","__label__""early departure fees"" early departur..."
1,rollaway bed rental,"__label__""rollawayscribs""","__label__""rollawayscribs"" rollaway bed rental"
2,pet charge,"__label__""pet fees""","__label__""pet fees"" pet charge"
3,pet cleaning fee,"__label__""pet fees""","__label__""pet fees"" pet cleaning fee"
4,banquet breakfast,"__label__""banquet food revenue breakfast""","__label__""banquet food revenue breakfast"" banq..."
...,...,...,...
529,room charge,"__label__""transient retail""","__label__""transient retail"" room charge"
530,city / county tax,"__label__""rooms city tax""","__label__""rooms city tax"" city / county tax"
531,state tax,"__label__""rooms sales tax""","__label__""rooms sales tax"" state tax"
532,debits,"__label__""guest ledger""","__label__""guest ledger"" debits"


In [ ]:
# Save train data
with open("dstools.train", "w") as file:
    for label_text in train["label_text"]:
        file.write(label_text + "\n")

# Save test data
with open("dstools.test", "w") as file:
    for label_text in test["label_text"]:
        file.write(label_text + "\n")


In [ ]:
!pip install fasttext

In [ ]:
import fasttext
fmodel = fasttext.load_model(r"/content/drive/MyDrive/cc.en.300.bin")


In [ ]:
import fasttext

model = fasttext.train_supervised(input="/content/dstools.train", epoch=100)


In [ ]:
model.test('/content/dstools.test')

(102, 0.9509803921568627, 0.9509803921568627)

In [ ]:
model.predict("early departure fee")

(('__label__"early',), array([0.85142648]))

In [ ]:
from gensim.models import FastText

# Assuming you have trained and used your FastText model for predictions
# `model` is your FastText model object

# Save the model to disk
model.save_model("fasttext_model.bin")

# Now you can load the model from disk later using:
# loaded_model = FastText.load_model("fasttext_model.bin")


In [ ]:
model.get_nearest_neighbors("banquet")

[(0.9997298121452332, 'market'),
 (0.9985095858573914, 'beverage"'),
 (0.9979128241539001, 'cmn'),
 (0.9979128241539001, 'campaig'),
 (0.9974956512451172, 'bs'),
 (0.9973974823951721, 'breakfast"'),
 (0.9973533153533936, 'sundries'),
 (0.996333658695221, 'r1'),
 (0.9959368109703064, 'breakfast'),
 (0.994982898235321, 'break"')]

In [ ]:
import csv

def process_csv(input_csv, output_csv):
    with open(input_csv, 'r', newline='') as input_file, \
            open(output_csv, 'w', newline='') as output_file:
        reader = csv.DictReader(input_file)
        fieldnames = reader.fieldnames

        # Add column names to the writer object
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()

        # Process each row and write to output CSV file
        for row in reader:
            # Modify the value in the "label" column
            row['label'] = "+ row['label'] +"
            writer.writerow(row)

# Input CSV file containing records
input_csv = ''

# Output CSV file to write processed records
output_csv = 'output.csv'

# Process CSV and write to output CSV file
process_csv(input_csv, output_csv)
